In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from sklearn.preprocessing import StandardScaler


In [ ]:
# Initialize Faker for synthetic data generation
fake = Faker("en_IN")
Faker.seed(0)
random.seed(0)


num_victims = 50  # Number of disaster victims
num_shelters = 3  # Number of available shelters in Thiruvananthapuram


# **Defining Shelter Locations**

In [ ]:
# Thiruvananthapuram
thiruvananthapuram_center = (8.5241, 76.9366)


shelters = [
    {'shelter_id': 1, 'shelter_name': "Shelter A", 'location': (8.5241, 76.9366)},
    {'shelter_id': 2, 'shelter_name': "Shelter B", 'location': (8.5331, 76.9492)},
    {'shelter_id': 3, 'shelter_name': "Shelter C", 'location': (8.5146, 76.9359)},
]


# **Random Location Generation for Victims**


In [ ]:
# Function to generate random coordinates within a small area around Thiruvananthapuram
def random_location(center, radius=0.02):
    lat_variation = random.uniform(-radius, radius)
    lon_variation = random.uniform(-radius, radius)
    return (round(center[0] + lat_variation, 6), round(center[1] + lon_variation, 6))

# Generate victim data at random locations within Thiruvananthapuram
victims = []
for victim_id in range(num_victims):
    current_location = random_location(thiruvananthapuram_center)
    victim_info = {
        'victim_id': victim_id,
        'victim_name': fake.name(),
        'current_location': current_location,
        'urgency_level': random.choice(['Low', 'Medium', 'High']),
        'priority_needs': random.choice(["Medical Attention", "Food & Water", "Rescue Priority"]),
        'contact_info': fake.phone_number()
    }
    victims.append(victim_info)

# Convert to DataFrames
victims_df = pd.DataFrame(victims)


# Display sample data
print("Victims DataFrame")
print(victims_df.head())



Victims DataFrame
   victim_id  victim_name       current_location urgency_level  \
0          0    Aarav Sen  (8.533763, 76.930037)           Low   
1          1  George Tank  (8.515335, 76.926205)        Medium   
2          2  Sai Chaudry  (8.528505, 76.943536)          High   
3          3  Jagat Wagle  (8.538251, 76.921894)        Medium   
4          4  Nirja Raval  (8.534039, 76.949756)           Low   

      priority_needs   contact_info  
0    Rescue Priority     7034148926  
1       Food & Water  +918640140046  
2    Rescue Priority     4543910144  
3       Food & Water  +912164482983  
4  Medical Attention     4475922116  


In [ ]:
shelters_df = pd.DataFrame(shelters)
print("\nShelters DataFrame")
print(shelters_df.head())


Shelters DataFrame
   shelter_id shelter_name           location
0           1    Shelter A  (8.5241, 76.9366)
1           2    Shelter B  (8.5331, 76.9492)
2           3    Shelter C  (8.5146, 76.9359)


# **Clustering Victims Based on Location**

In [ ]:
# Clustering victims based on current location
location_data = np.array(victims_df['current_location'].tolist())
scaler = StandardScaler()
location_data_scaled = scaler.fit_transform(location_data)

# DBSCAN for spatial clustering
db = DBSCAN(eps=0.5, min_samples=2).fit(location_data_scaled)
victims_df['cluster_id'] = db.labels_

# Display clustered victims (ignoring -1 labels, which indicate outliers)
clustered_victims = victims_df[victims_df['cluster_id'] != -1]
print("\nClustered Victims")
print(clustered_victims.head())



Clustered Victims
   victim_id      victim_name       current_location urgency_level  \
0          0        Aarav Sen  (8.533763, 76.930037)           Low   
1          1      George Tank  (8.515335, 76.926205)        Medium   
2          2      Sai Chaudry  (8.528505, 76.943536)          High   
4          4      Nirja Raval  (8.534039, 76.949756)           Low   
5          5  Upadhriti Dewan  (8.527883, 76.944543)           Low   

      priority_needs   contact_info  cluster_id  
0    Rescue Priority     7034148926           0  
1       Food & Water  +918640140046           0  
2    Rescue Priority     4543910144           1  
4  Medical Attention     4475922116           2  
5  Medical Attention     4576377857           1  


# **Calculating Distances to Shelters**

In [ ]:
# Calculate distances to the nearest shelters
def calculate_distances(victims_df, shelters_df):
    distances = []
    for _, victim in victims_df.iterrows():
        current_location = victim['current_location']
        shelter_locations = shelters_df['location'].tolist()
        for shelter_location in shelter_locations:
            distance = great_circle(current_location, shelter_location).kilometers
            distances.append({'victim_id': victim['victim_id'],
                              'shelter_id': shelters_df.loc[shelters_df['location'] == shelter_location, 'shelter_id'].values[0],
                              'distance': distance})
    return pd.DataFrame(distances)

distances_df = calculate_distances(clustered_victims, shelters_df)

# Sort by victim id and distance to find the nearest shelter
sorted_distances = distances_df.sort_values(by=['victim_id', 'distance'])
print("\nDistances to Nearest Shelters")
print(sorted_distances)



Distances to Nearest Shelters
     victim_id  shelter_id  distance
0            0           1  1.294356
1            0           2  2.108531
2            0           3  2.226236
5            1           3  1.069281
3            1           1  1.502202
..         ...         ...       ...
138         48           1  1.753490
139         48           2  3.260787
143         49           3  2.270016
141         49           1  2.833821
142         49           2  3.093191

[144 rows x 3 columns]


# **Grouping Victims into Transport Units**

In [ ]:
# Grouping victims into transport units based on proximity to shelters
transport_capacity = 5

# Group by shelter and victim ID, then assign transport groups
sorted_distances['transport_group'] = (sorted_distances.groupby('shelter_id').cumcount() // transport_capacity) + 1

# Merging to get victim names
final_transport_groups = sorted_distances.merge(victims_df[['victim_id', 'victim_name']], on='victim_id')
print("\nFinal Transport Assignments")
print(final_transport_groups[['victim_name', 'shelter_id', 'distance', 'transport_group']])



Final Transport Assignments
         victim_name  shelter_id  distance  transport_group
0          Aarav Sen           1  1.294356                1
1          Aarav Sen           2  2.108531                1
2          Aarav Sen           3  2.226236                1
3        George Tank           3  1.069281                1
4        George Tank           1  1.502202                1
..               ...         ...       ...              ...
139    Yauvani Bandi           1  1.753490               10
140    Yauvani Bandi           2  3.260787               10
141  Dakshesh Butala           3  2.270016               10
142  Dakshesh Butala           1  2.833821               10
143  Dakshesh Butala           2  3.093191               10

[144 rows x 4 columns]


In [ ]:
import matplotlib.pyplot as plt
import folium
from folium import plugins

# Create a base map centered around Thiruvananthapuram
m = folium.Map(location=[8.5, 76.9], zoom_start=12)

# Add shelters to the map
for idx, shelter in shelters_df.iterrows():
    folium.Marker(
        location=shelter['location'],
        popup=f"Shelter: {shelter['shelter_name']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Add victims to the map
for idx, victim in victims_df.iterrows():
    folium.Marker(
        location=victim['current_location'],
        popup=f"Victim: {victim['victim_name']}, Urgency: {victim['urgency_level']}, Need: {victim['priority_needs']}",
        icon=folium.Icon(color='blue', icon='user')
    ).add_to(m)

# Create a second map for the carpool lines
m_carpool = folium.Map(location=[8.5, 76.9], zoom_start=12)

# Plot the lines connecting victims to their assigned shelters
for group, group_data in sorted_distances.groupby('transport_group'):
    color = plt.cm.viridis(group / len(sorted_distances['transport_group'].unique()))  # Using a color map
    for _, row in group_data.iterrows():
        victim_location = victims_df.loc[victims_df['victim_id'] == row['victim_id'], 'current_location'].values[0]
        shelter_location = shelters_df.loc[shelters_df['shelter_id'] == row['shelter_id'], 'location'].values[0]

        folium.PolyLine(
            locations=[victim_location, shelter_location],
            color=color,
            weight=2,
            opacity=0.6
        ).add_to(m_carpool)

# Add shelters to the carpool map
for idx, shelter in shelters_df.iterrows():
    folium.Marker(
        location=shelter['location'],
        popup=f"Shelter: {shelter['shelter_name']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m_carpool)

# Add victims to the carpool map
for idx, victim in victims_df.iterrows():
    folium.Marker(
        location=victim['current_location'],
        popup=f"Victim: {victim['victim_name']}, Urgency: {victim['urgency_level']}, Need: {victim['priority_needs']}",
        icon=folium.Icon(color='blue', icon='user')
    ).add_to(m_carpool)

# Save the maps to HTML files
m.save('victims_and_shelters_map.html')
m_carpool.save('carpool_connections_map.html')

# Displaying the maps in Jupyter Notebook (if applicable)
m




In [ ]:
import folium
import numpy as np
import matplotlib.pyplot as plt

# Create a base map centered around Thiruvananthapuram
m_carpool = folium.Map(location=[8.5, 76.9], zoom_start=12)

# Function to generate distinct colors for the lines
def generate_colors(num_colors):
    # Generate a list of colors using matplotlib
    colors = plt.cm.viridis(np.linspace(0, 1, num_colors))
    # Convert colors to hex format
    hex_colors = ['#%02x%02x%02x' % (int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in colors]
    return hex_colors

# Get unique transport groups and generate colors
unique_groups = sorted_distances['transport_group'].unique()
colors = generate_colors(len(unique_groups))

# Plot the lines connecting victims to their assigned shelters with different colors for each group
for group_idx, group in enumerate(unique_groups):
    group_data = sorted_distances[sorted_distances['transport_group'] == group]

    for _, row in group_data.iterrows():
        victim_location = victims_df.loc[victims_df['victim_id'] == row['victim_id'], 'current_location'].values[0]
        shelter_location = shelters_df.loc[shelters_df['shelter_id'] == row['shelter_id'], 'location'].values[0]

        folium.PolyLine(
            locations=[victim_location, shelter_location],
            color=colors[group_idx],  # Use the color for the current group
            weight=4,  # Make the line thicker
            opacity=0.8,  # Slightly more opaque
            dash_array='5, 5'  # Optional: makes the line dashed for better visibility
        ).add_to(m_carpool)

# Add shelters to the carpool map
for idx, shelter in shelters_df.iterrows():
    folium.Marker(
        location=shelter['location'],
        popup=f"Shelter: {shelter['shelter_name']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m_carpool)

# Add victims to the carpool map
for idx, victim in victims_df.iterrows():
    folium.Marker(
        location=victim['current_location'],
        popup=f"Victim: {victim['victim_name']}, Urgency: {victim['urgency_level']}, Need: {victim['priority_needs']}",
        icon=folium.Icon(color='blue', icon='user')
    ).add_to(m_carpool)

# Create a legend for carpool groups
legend_html = '''
<div style="position: fixed;
             bottom: 50px; left: 50px; width: 200px; height: auto;
             border:2px solid grey; z-index:9999; font-size:14px;
             background-color: white;">
&nbsp; <b>Carpool Group Legend</b> <br>
'''
for idx, group in enumerate(unique_groups):
    legend_html += f'&nbsp; <i style="background:{colors[idx]}; width: 20px; height: 20px; display: inline-block;"></i> Group {group}<br>'

legend_html += '</div>'
m_carpool.get_root().html.add_child(folium.Element(legend_html))

# Save the carpool map with connections
m_carpool.save('carpool_connections_with_victims_legend.html')

# Display the map (if in Jupyter Notebook)
m_carpool

